In [1]:
import numpy as np
from collections import defaultdict
import random 
import sys
import time

In [5]:
class K_Means:
    def __init__ (self, K, centroid_difference, max_iters):
        self.K = K
        self.centroid_difference = centroid_difference
        self.max_iters = max_iters
    
    def get_data(self, path_tfidfs, path_idfs, isTrained):
        
        # sparse to dense
        def sparse_to_dense(sparsed_doc, vocab_size):
            densed_doc = np.zeros(vocab_size)
            split_sparsed_doc = sparsed_doc.split()
            for item in split_sparsed_doc:
                idx = int(item.split(':')[0])
                tfidf = float(item.split(':')[1])
                densed_doc[idx] = tfidf
            return densed_doc
        
        # get vocab_size
        with open(path_idfs) as f:
            vocab_size = len(f.read().splitlines())
            
        # get tfidf of each doc and transform from sparse vetorto dense vector:
        with open(path_tfidfs) as f:
            doc_lines = f.read().splitlines()
        if isTrained == True:
            self.data = []      # each doc is an item of data
        else:
            self.data_test = []
            
        for doc in doc_lines:
            features = doc.split('<fff>')
            label = int(features[0])
            doc_id = int(features[1])
            dense_doc = sparse_to_dense(features[2], vocab_size)
            
            if isTrained == True:
                self.data.append((label, dense_doc))    
            else:
                self.data_test.append((label, dense_doc))
                
            
    def initialize(self):    # k-means++ initialization
        self.centroids = {}    # a dict to store K centroids; key of this dict to identify a centroid belong to which cluster
        # randomly pick one data point as the first centroid
        self.centroids[0] = self.data[random.randrange(0, len(self.data)-1, 1)][1]
        idx = 0
        d = sys.maxsize
        # a list in which each item is the distance between a data point and the nearest previously chosen centroid 
        dist_points_centroid = [d]*len(self.data)
        dist_points_centroid = np.array(dist_points_centroid)
        
        # loop to find K-1 centroids
        for centroid_idx in range(self.K - 1):
            for i in range(len(self.data)):
                point = self.data[i][1]
                # only need to compute distance between the point and the newest centroid
                temp_dist = np.sum((self.centroids[idx] - point)**2)
                dist_points_centroid[i] = min(dist_points_centroid[i], temp_dist)
          
            # choose the data point that has the largest distance with its nearest centroid
            next_centroid = self.data[np.argmax(dist_points_centroid)][1]
            self.centroids[idx+1] = next_centroid
            idx += 1
                
                
    def assign_example(self, example):       # func to assign one doc to its cluster
        doc = example[1]
        dist = sys.maxsize
        class_idx = -1
        for j in self.centroids:
            temp_dist = np.sum((doc - self.centroids[j])**2)
            if (dist > temp_dist):
                class_idx = j
                dist = temp_dist
        self.clusters[class_idx].append(example)
    
    def fit(self, data_set):
    
        # begin iterations
        for i in range(self.max_iters):
            
            self.clusters = {}
            for i in range(self.K):
                self.clusters[i] = []    # list of (actual label - doc) belonging to this clusters
        
            for item in data_set:
                self.assign_example(item)
            
            list_previous_centroids = [list(centroid) for centroid in self.centroids.values()]
            
            # caculate new centroids
            for cluster_idx in self.clusters:
                if len(self.clusters[cluster_idx]) > 0:
                    doc_list = [item[1] for item in self.clusters[cluster_idx]]
                    aver_doc = np.mean(doc_list, axis=0)
                    denom = np.sqrt(np.sum(aver_doc ** 2))
                   # assert denom > 0
                    new_centroid = np.array([numerator / denom for numerator in aver_doc])
                    self.centroids[cluster_idx] = new_centroid

            isOptimal = False
            
            list_current_centroids = [list(centroid) for centroid in self.centroids.values()]
    
            # difference between old centroids and new centroids
            minus_centroids = [centroid for centroid in list_current_centroids if centroid not in list_previous_centroids]
            if len(minus_centroids) <= self.centroid_difference:
                isOptimal = True
            if isOptimal == True:
                break
      
    def predict(self, data_set):
        for i in range(self.K):
            self.clusters[i] = []  
        for item in data_set:
            self.assign_example(item)
    
    def compute_purity(self, data_set, justFit):
        if justFit == False:
            self.predict(data_set)    # predict data_set to get its centroids and clusters
        
        count = 0
        for i in self.clusters:
            labels_of_cluster = [member[0] for member in self.clusters[i]]
            count_max_label = max([labels_of_cluster.count(label) for label in range(20)])
            count += count_max_label
        return count * 1. / len(data_set)
    
        

In [1]:
#print(len(km.data_test))
#km.data[11313][1]


In [6]:
km = K_Means(20, 10, 100)
km.get_data("D:\\movedFromC\\123\\20192\\PRJ2\\Project2\\words_tfidfs.txt", "D:\\movedFromC\\123\\20192\\PRJ2\\Project2\\words_idfs.txt", True)
km.get_data("D:\\movedFromC\\123\\20192\\PRJ2\\Project2\\words_tfidfs_test.txt", "D:\\movedFromC\\123\\20192\\PRJ2\\Project2\\words_idfs.txt", False)



In [7]:
best_model = km
tmp_training_purity = 0
best_training_purity  = 0

for i in range(5):
    t1 = time.time()
    km.initialize()
    km.fit(km.data)
    t2 = time.time()
    print("training time: ", t2-t1)
    tmp_training_purity = km.compute_purity(km.data, True)
    if (tmp_training_purity > best_training_purity):
        best_model = km
    print("purity of training set: ", tmp_training_purity)
    print("-------------------")

training time:  355.6602427959442
purity of training set:  0.39437864592540217
-------------------
training time:  515.011504650116
purity of training set:  0.4102881385893583
-------------------
training time:  626.3926403522491
purity of training set:  0.3580519710093689
-------------------
training time:  613.0481243133545
purity of training set:  0.3975605444581934
-------------------
training time:  520.6776068210602
purity of training set:  0.42301573272052323
-------------------


In [8]:
print("bestt model")
print("purity of tesing set: ", best_model.compute_purity(best_model.data_test, False))

bestt model
purity of tesing set:  0.3996282527881041


11314